# DBX-Patch Quick Setup

This notebook installs and configures dbx-patch to enable editable package imports in Databricks.

**Run all cells in order.**


# Install uv package manager


In [ ]:
%pip install -q uv

# Verify installation


In [ ]:
!uv --version

In [ ]:
import os

os.environ["DBX_PATCH_DEBUG"] = "1"
os.environ["DBX_PATCH_LOG_LEVEL"] = "DEBUG"

## Step 1: Install dbx-patch


In [ ]:
# Install dbx-patch from PyPI
# %pip install -q dbx-patch
!cd .. && uv build && uv pip install ./dist/dbx_patch-1.0.0-py3-none-any.whl

# Verify installation
import dbx_patch

print(f"\n✅ dbx-patch {dbx_patch.__version__} installed successfully")

## Step 2: Install sitecustomize.py (Recommended)


In [ ]:
# Install sitecustomize.py for automatic patching on kernel startup
from dbx_patch import install_sitecustomize

install_sitecustomize()

# Python kernel will restart automatically if running in Databricks
# If not, you'll need to manually restart the kernel

## Step 3: Verify Installation (After Kernel Restart)


In [ ]:
# Check sitecustomize.py status
from dbx_patch import check_sitecustomize_status

status = check_sitecustomize_status()

print("\n" + "=" * 80)
if status.is_dbx_patch:
    print("✅ sitecustomize.py is active - patches will apply automatically!")
else:
    print("⚠️  sitecustomize.py not detected - you may need to restart kernel")
    print("   Or install it manually: install_sitecustomize()")

## Step 4: Verify Patches Are Applied


In [ ]:
# Verify that all patches are applied
from dbx_patch import verify_editable_installs

result = verify_editable_installs()

print("\n" + "=" * 80)
print(f"Status: {result.status.upper()}")
print(f"Editable paths detected: {len(result.editable_paths)}")

if result.editable_paths:
    print("\nEditable install paths:")
    for path in result.editable_paths:
        print(f"  • {path}")
else:
    print("\n⚠️  No editable installs detected yet.")
    print("   Install packages with: %pip install -e /path/to/package")

## Step 5: Install Your Editable Package


In [ ]:
# Example: Install the test package from this repo
%pip install -e ../test_package

# Or install your own package:
# %pip install -e /Workspace/Repos/your-repo/your-package
# %pip install -e /Workspace/Users/me@company.com/my-project

print("\n✅ Package installed in editable mode")

## Step 6: Test Import


In [ ]:
# Try importing the test package
try:
    from test_package import hello  # ty:ignore[unresolved-import]

    print("✅ Import successful!")
    hello()
except ImportError as e:
    print(f"❌ Import failed: {e}")
    print("\nTroubleshooting:")
    print("  1. Make sure you ran Step 5 to install the package")
    print("  2. Check that patches are applied (Step 4)")
    print("  3. Try restarting the kernel and re-running from Step 3")

## Alternative: Manual Patching (Not Recommended)

If you can't use sitecustomize.py, you can manually apply patches:


In [ ]:
# Apply patches manually (must run in every notebook)
from dbx_patch import patch_dbx

result = patch_dbx(force_refresh=False)

print(f"\n{'=' * 80}")
print(f"Patches applied: {result.overall_success}")
print(f"Editable paths found: {len(result.editable_paths)}")

if result.editable_paths:
    for path in result.editable_paths:
        print(f"  • {path}")

## Troubleshooting

If imports still fail:

1. **Run the diagnostic notebook** for detailed analysis:

   ```
   notebooks/diagnostic_editable_imports.ipynb
   ```

2. **Check sitecustomize.py status**:

   ```python
   from dbx_patch import check_sitecustomize_status
   check_sitecustomize_status(verbose=True)
   ```

3. **Verify patches are applied**:

   ```python
   from dbx_patch import check_patch_status
   check_patch_status(verbose=True)
   ```

4. **Check the [Solution Guide](../docs/docs/files/solution-guide.md)** for more help

## Next Steps

✅ **Setup complete!** You can now:

- Install packages in editable mode: `%pip install -e /path/to/package`
- Import them directly: `from your_package import your_function`
- Edit source code and see changes immediately with autoreload

For more details, see:

- [Quickstart Guide](quickstart.ipynb) - Complete usage examples
- [Getting Started](../docs/docs/files/getting-started.md) - Full documentation
